In [ ]:
import numpy as np
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()

movieProperties = ratings.groupby('movie_id').agg({'rating': ['size', 'mean']})
movieProperties.head()

movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()
movieDict = {}
with open(r'u.item') as f:
    temp = ''
    for line in f:
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?

In [49]:
import operator

def ComputeDistance(movieA, movieB):
    # Aplica a distancia euclidiana entre os generos dos filmes
    genre_distance = np.linalg.norm(movieA[1] - movieB[1])
    # Calcula a diferença entre a popularidade dos filmes
    popularity_diff = abs(movieA[2] - movieB[2])
    # Calcula a diferença entre as avaliações dos filmes
    rating_diff = abs(movieA[3] - movieB[3])
    return genre_distance + popularity_diff + rating_diff

def getNeighbors(movieID, K, distance_metric):
    distances = []
    for movie in movieDict:
        if movie != movieID:
            dist = distance_metric(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
neighbors = getNeighbors(1, K, ComputeDistance)
avgRating = sum([movieDict[neighbor][3] for neighbor in neighbors]) / K
for neighbor in neighbors:
    print(movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))

Aladdin (1992) 3.8127853881278537
Full Monty, The (1997) 3.926984126984127
Winnie the Pooh and the Blustery Day (1968) 3.8
Raising Arizona (1987) 3.875
Aladdin and the King of Thieves (1996) 2.8461538461538463
Pinocchio (1940) 3.6732673267326734
Monty Python and the Holy Grail (1974) 4.0664556962025316
Fish Called Wanda, A (1988) 3.785425101214575
Willy Wonka and the Chocolate Factory (1971) 3.6319018404907975
Grand Day Out, A (1992) 4.106060606060606


In [50]:
avgRating

3.752403393196701

In [46]:
movieDict[1][3]

3.8783185840707963